In [65]:
import sys
sys.path.append("/home/suhang/jupyter/jupyter_env/lib/python3.11/site-packages")
import gzip
# import pyfrost
import gzip
from collections import defaultdict
import json
import numpy
from Bio import SeqIO

In [101]:
reference_file = "./NC_012920.1.fasta"
reference_record = SeqIO.parse(reference_file, "fasta")
for record in reference_record:
    reference_id = record.id
    reference_sequence = str(record.seq)


In [102]:
# def get_reference_kmer_profile(reference_sequence, k):
#     ref_kmer_profile = {}
#     r = len(reference_sequence) - k + 1
#     for i in range(r):
#         kmer = reference_sequence[i:i+k]
#         ref_kmer_profile[kmer] = ref_kmer_profile.get(kmer, 0) + 1


#     unique_kmers = []
#     for kmer, count in ref_kmer_profile.items():
#         if count == 1:
#             unique_kmers.append(kmer)
#     return unique_kmers


In [126]:
def map_to_genome(contig, k):
    PositionDict = defaultdict(list)
    for i in range(len(contig) - k + 1):
        kmer = contig[i:i+k]
        PositionDict[kmer] = PositionDict.get(kmer, []) + [i]
    return PositionDict

def create_anchors(PositionDict, k):
    anchor_updated_list = []

    for kmer in PositionDict:
        if len(PositionDict[kmer]) == 1:
            anchor_updated_list.append(kmer)
    AnchorInfo = {}
    for kmer in anchor_updated_list:
        pos = PositionDict[kmer][0]
        anchorname = "A%06d" %  (pos)
        AnchorInfo[anchorname] = {}
        AnchorInfo[anchorname]['seq'] = kmer
        AnchorInfo[anchorname]['pos'] = pos
    return AnchorInfo

def get_final_anchor(anchor_info, k):
    final_anchor = {}
    anchor_names = sorted(anchor_info.keys())
    
    anchor_unadjacent_list = []
    last_position = 0
    
    for anchor in anchor_names:
        position = anchor_info[anchor]['pos']
        if position > last_position + k:
            anchor_unadjacent_list.append(anchor)
            last_position = position
    
    for anchor_name in anchor_unadjacent_list:
        if anchor_name in anchor_info:
            final_anchor[anchor_name] = anchor_info[anchor_name]
    
    return final_anchor

In [127]:
# PositionDict

In [144]:
k = 21
# unique_kmers = get_reference_kmer_profile(reference_sequence, k)
PositionDict = map_to_genome(reference_sequence, k)
AnchorInfo = create_anchors(PositionDict, k)

print(len(AnchorInfo))
Final_anchor = get_final_anchor(AnchorInfo, k)
print(len(Final_anchor))

16549
752


In [145]:
Final_anchor

{'A000022': {'seq': 'TAACCACTCACGGGAGCTCTC', 'pos': 22},
 'A000044': {'seq': 'ATGCATTTGGTATTTTCGTCT', 'pos': 44},
 'A000066': {'seq': 'GGGGGTATGCACGCGATAGCA', 'pos': 66},
 'A000088': {'seq': 'TGCGAGACGCTGGAGCCGGAG', 'pos': 88},
 'A000110': {'seq': 'ACCCTATGTCGCAGTATCTGT', 'pos': 110},
 'A000132': {'seq': 'TTTGATTCCTGCCTCATCCTA', 'pos': 132},
 'A000154': {'seq': 'TATTTATCGCACCTACGTTCA', 'pos': 154},
 'A000176': {'seq': 'TATTACAGGCGAACATACTTA', 'pos': 176},
 'A000198': {'seq': 'TAAAGTGTGTTAATTAATTAA', 'pos': 198},
 'A000220': {'seq': 'GCTTGTAGGACATAATAATAA', 'pos': 220},
 'A000242': {'seq': 'AATTGAATGTCTGCACAGCCA', 'pos': 242},
 'A000264': {'seq': 'TTTCCACACAGACATCATAAC', 'pos': 264},
 'A000286': {'seq': 'AAAAATTTCCACCAAACCCCC', 'pos': 286},
 'A000308': {'seq': 'CTCCCCCGCTTCTGGCCACAG', 'pos': 308},
 'A000330': {'seq': 'ACTTAAACACATCTCTGCCAA', 'pos': 330},
 'A000352': {'seq': 'CCCCAAAAACAAAGAACCCTA', 'pos': 352},
 'A000374': {'seq': 'CACCAGCCTAACCAGATTTCA', 'pos': 374},
 'A000396': {'seq'

In [148]:
# other_finalanchor = "./anchors.json"
# with open(other_finalanchor, 'r') as fp:
#     data = json.load(fp)
# len(data), len(AnchorInfo)
# print(data == Final_anchor)
# print(sorted([data[anchor]["pos"] for anchor in data]))
# # anchorname = sorted(data.keys())
# # poslist = []
# # for anchor in anchorname:
# #     poslist.append(data[anchor]['pos'])
# # poslist = numpy.array(poslist)
# # numpy.all((poslist[1:] - poslist[:-1]) > k)
# # poslist
# sorted([Final_anchor[anchor]["pos"] for anchor in Final_anchor])
# for anchor, D in data.items():
#     print(Final_anchor[anchor], D)

In [132]:
# Final_anchor = {anchor:AnchorInfo[anchor] for anchor in anchor_unadjacent_list}
# print(len(Final_anchor))

In [133]:
def reverse_complement(kmer):
    return ''.join([{'A':'T','C':'G','G':'C','T':'A', "N":"N"}[base] for base in reversed(kmer)])


In [134]:
# create edge
def mapping_info(AnchorInfo, contig, k):
    anchor_list = list(AnchorInfo.keys())
    Anchorseq = {AnchorInfo[anchor]['seq']:anchor for anchor in anchor_list}

    seqlist = Anchorseq.keys()
    PositionDict = defaultdict(list)
    for anchor_seq in seqlist:
        anchor_rev = reverse_complement(anchor_seq)
        PositionDict[anchor_seq]
        PositionDict[anchor_rev]

    for i in range(1, len(contig) - k + 1):
        kmer = contig[i:i+k]
        if kmer in PositionDict:
            PositionDict[kmer] = PositionDict.get(kmer, []) + [i]
            
    A = {}
    SVs = {}
    for anchor, D in AnchorInfo.items():
        anchor_seq = D['seq']
        anchor_rev = reverse_complement(anchor_seq)
        poslist = PositionDict[anchor_seq] + PositionDict[anchor_rev]
        if len(poslist) == 1:
            A[anchor] = poslist[0]
        else:
            SVs[anchor] = poslist
            
    return A, SVs

def find_edge_info(src_pos, dst_pos, k, contig, contigname, sample, Anchorseq):
    E = {} # edgeinfo
    # get source infomation
    if src_pos == 0:
        src = "SOURCE"
        src_seq = ""
        pr = False
    else:
        src_seq = contig[src_pos:src_pos + k]
        try:
            src = Anchorseq[src_seq]
            pr = False
        except:
            src_seq = reverse_complement(src_seq)
            src = Anchorseq[src_seq]
            pr = True

    dst_seq = contig[dst_pos:dst_pos+k]
    
    if dst_pos == len(contig): # fix sink edge issue 08/29/23
        dst = "SINK"
        dst_seq = ""
        sr = True
    else:
        try:
            dst = Anchorseq[dst_seq]
            sr = False
        except:
            dst_seq = reverse_complement(dst_seq)
            dst = Anchorseq[dst_seq]
            sr = True
            
    if src_pos == 0:
        edge_seq = contig[src_pos:dst_pos] # first edge fix bug 08/28/2023
    else:
        edge_seq = contig[src_pos+k:dst_pos] # fix bug 08/23/2023
        
    if pr and sr:
        edge_seq = reverse_complement(edge_seq)
        node = src
        src = dst
        dst = node     
    E = {}
    E['seq'] = edge_seq
    E['src'] = src
    E['dst'] = dst
    E['reads'] = [contigname]
    E['strain'] = [sample]
    return E


# loop through all reads
def create_edgefile(read_file, AnchorInfo, k):
    Edge_info = {}
    Outgoing = {}
    
    edgenum_perread = []
    anchor_list = list(AnchorInfo.keys())
    Anchorseq = {AnchorInfo[anchor]['seq']:anchor for anchor in anchor_list}
    
    read_records = SeqIO.parse(read_file, "fasta")
    contig_index = 0
    for record in read_records:
        contig_index += 1
        contig_name = record.id
        contig = str(record.seq)
        sample_name = contig_name.split('|')[-1]
        A, SVs = mapping_info(AnchorInfo, contig, k)
        splitposlist = sorted(A.values())
        edgeindex = 0 # 
        src_pos = 0
        for dst_pos in splitposlist:
            E = find_edge_info(src_pos, dst_pos, k, contig, contig_name, sample_name, Anchorseq)
            src = E['src']
            edgelist = Outgoing.get(src, [])
            for edge in edgelist:
                if Edge_info[edge]['dst'] != E['dst']:
                    continue
                if Edge_info[edge]['seq'] == E['seq']:
                    Edge_info[edge]['reads'] += E['reads']
                    Edge_info[edge]['strain'] = sorted(set(Edge_info[edge]['strain']) | set(E['strain']))
                    break
            else:
                edgename = "E%05d.%04d" % (contig_index, edgeindex)
                Edge_info[edgename] = E
                Outgoing[src] = Outgoing.get(src,[]) + [edgename]
                edgeindex += 1
            # update
            src_pos = dst_pos
        dst_pos = len(contig) # fix bug 08/29/2023 
        E = find_edge_info(src_pos, dst_pos, k, contig, contig_name, sample_name, Anchorseq)
        src = E['src']
        edgelist = Outgoing.get(src, [])
        for edge in edgelist:
            if Edge_info[edge]['dst'] != E['dst']:
                continue
            if Edge_info[edge]['seq'] == E['seq']:
                Edge_info[edge]['reads'] += E['reads']
                Edge_info[edge]['strain'] = sorted(set(Edge_info[edge]['strain']) | set(E['strain']))
                break
        else:
            edgename = "E%05d.%04d" % (contig_index, edgeindex)
            Edge_info[edgename] = E
            Outgoing[src] = Outgoing.get(src,[]) + [edgename]
            edgeindex += 1
        
        edgenum_perread.append(edgeindex)   
    return Edge_info, Outgoing
        #print(contig_name, edgeindex)


In [135]:
read_file = "./HG00621_for_hidive.fasta"
%time Edge_info, Outgoing = create_edgefile(read_file, Final_anchor, k)

CPU times: user 3.66 s, sys: 61.7 ms, total: 3.73 s
Wall time: 3.77 s


In [136]:
def write_gfa(AnchorInfo, Edge_info, outputfilename):
    header = ['H\tVN:Z:1.0\n']
    # add json annotation to edge and anchor sequences
    AnchorS = []
    for anchor,D in AnchorInfo.items():
        seq = D.pop('seq')
        json_string = json.dumps(D)
        AnchorS += ['S\t%s\t%s\t%s\n' % (anchor, seq, "PG:J:" + json_string)]
   
    EdgeS = []   
    Link = [] 
    for edge,edge_dict in Edge_info.items():
        seq = edge_dict.pop('seq')
        src = edge_dict.pop('src')
        dst = edge_dict.pop('dst')
        
        json_string = json.dumps(edge_dict)
        EdgeS += ['S\t%s\t%s\t%s\t%s\n' % (edge, seq, "PG:J:" + json_string, "RC:i:" + str(len(edge_dict['reads'])))]
        Link.append('L\t%s\t%s\t%s\t%s\t%s\n'% (src, "+", edge, "+", "0M"))
        Link.append('L\t%s\t%s\t%s\t%s\t%s\n'% (edge, "+", dst, "+", "0M"))     

    with open(outputfilename, 'w') as fp:
        for h in header:
            fp.write(h)
        for s in AnchorS:
            fp.write(s)
        for s in EdgeS:
            fp.write(s)
        for l in Link:
            fp.write(l)

In [137]:
filename = 'test.gfa'
write_gfa(Final_anchor, Edge_info, filename)